In [ ]:
!nvidia-smi

In [ ]:
%pip install -U accelerate peft bitsandbytes transformers trl datasets fsspec

# [PEFT - Parameter-Efficient Fine-Tuning](https://github.com/huggingface/peft)

- 딥러닝 모델의 크기가 커질수록 태스크별로 모든 파라미터를 조정하는 full-fine-tuning 방식은 자원 소모와 시간, 비용적인 측면에서 점점 힘들어 지고 있다.  
  - 예: 대규모 언어 모델(LLM)에서 full-fine-tuning 시 수십억 개에서 수천억 개의 파라미터를 업데이트해야 함.  
- 이를 해결하기 위해 PEFT(Parameter-Efficient Fine-Tuning) 방법이 주목받고 있다.  
  - PEFT는 다운스트림 작업의 성능을 유지하거나 향상시키면서 계산량과 모델 크기를 줄이는 것이 목표다.  
  - 예: LoRA(Low-Rank Adaptation), Adapter Tuning 같은 기술이 대표적인 PEFT 방식이다.  

## 주요 기법

1. **어댑터 튜닝(Adapter Tuning)**
    - 기존 사전학습 모델에 어댑터 레이어를 추가하고, 어댑터 레이어만 학습시키는 방식.
    - 이를 통해 모델의 주요 파라미터를 수정하지 않으면서도 새로운 작업에 맞게 미세 조정할 수 있다.
    - LoRA(Low-Rank Adaptation)이 대표적인 어댑터 튜닝 방식이다.
  
2. **프롬프트 튜닝(Prompt Tuning)**
    - 모델의 입력 앞에 학습 가능한 프롬프트를 추가하여 해당 프롬프트만 학습하는 방식.
    - 이를 통해 기존 모델의 파라미터를 수정하지 않고도 다양한 작업에 빠르게 적응할 수 있다.

3. P-튜닝(P-Tuning)
    - 입력 시퀀스 전체에 학습 가능한 연속적인 프롬프트 벡터를 삽입하여 해당 벡터들만 학습하는 방식.
    - 프롬프트 튜닝과 달리, P-튜닝은 입력의 다양한 위치에 프롬프트 벡터를 삽입하여 모델의 출력을 조정한다.



# [LoRA (Low-Rank Adaptation)](https://arxiv.org/abs/2106.09685)
- LoRA는 모델 파라미터를 재구성해 더 적은 파라미터를 학습하여 GPU 메모리 사용량을 줄인다. 
- LoRA의 파라미터 재구성은 모델의 Parameter 행렬을 더 작은 2개의 행렬 곱으로 표현해 추가하여 전체 파라미터를 수정(학습) 하는 것이 아니라 더 작은 2개 행렬을 수정(학습)하는 것을 말한다.

![LoRA](figures/LoRA.png)

- 위의 그림을 보면 파란색이 사전학습 모델의 파라미터고 주황색이 재구성된 파라미터 이다.
- 모델은 d차원의 입력 X와 `d x d` 차원의 파라미터 W를 곱해 최종적으로 d차원의 결과 h를 출력한다.
    - 만약 d차원이 100이라면 파라미터는 100000 개의 파라미터를 가진다. 
- LoRA는 모델의 파라미터를 고정시키고 **d x r**인 행렬 A와 **r x d** 인 행렬 B를 추가하고 이 두 행렬을 곱해 d차원의 결과 h`를 출력한다.
- 사전학습 모델의 파라미터가 출력한 h와 LoRA adapter가 출력한 h` 를 더해 최종 결과를 출력한다.
    - **d x r**인 행렬 A와 **r x d** 행렬 B를 행렬곱을 하면 **d x d** 행렬이 되어 사전학습 모델의 파라미터의 출력과 같은 차원의 결과를 출력한다.
- 행렬 A와 B의 r을 d보다 작은 값으로 설정하면 d x d 인 행렬 W 보다 훨씬 적은 파라미터로 학습시킬 수 있다.
- Adaptor가 추가 되어 전체 모델의 파라미터는 조금 늘어난다. 하지만 학습 과정에서는 adpator만 학습하므로 gradient와 옵티마이저의 파라미터 양이 줄어들어 GPU 메모리를 적세 사용할 수있다.

### LoRA 설정
- r: 행렬 A와 B를 만들때 r을 몇으로 할지 설정한다.
- alpah: adaptor 파라미터의 결과를 기존 파라미터의 결과에 얼마나 반영할지를 결정하는 하이퍼파라미터.
    - alpha/r 을 결과 d에 곱해 기존 파라미터의 결과와 더한다. 
        - alpha가 16이고 r이 8인 경우 행렬 A, B에 2(16/8)을 곱해 기존 파라미터에 더해준다.
    - alpha가 커질 수록 새로게 학습한 파라미터의 중요성을 더 크게 고려한다.
- 모델의 파라미터어 중 어디에 adaptor를 적용해 파라미터를 재구성할지 결정해야 한다.
    - transformer의 self-attention layer는 query, key, value, feed-forward의 linear layer로 구성되있다. 이 중 특정 파라미터에만 LoRA를 적용할 수있고 전체에 적용할 수있다.

# SQuAD Dataset
- The Stanford Question Answering Dataset. 스탠포드 대학에서 만든 10만개의 Question & Answer 쌍으로 구성된 데이터셋
- Question과 Answer를 위한 context 문장이 주어지고 해당 Paragraph에있는 정보를 통대로 정답을 맞추는 데이터셋이다.
- https://rajpurkar.github.io/SQuAD-explorer/
  
# KorQuAD Dataset
- Korean Question Answering Dataset (KorQuad v1.0)은 LG CNS에서 구축한 SQuAD 데이터셋의 한국어 버전.
- KorQuAD 1.0의 전체 데이터는 1,560 개의 Wikipedia article에 대해 10,645건의 Context 문장과 66,181 개의 질의응답 쌍으로, Training set 60,407 개, validation set
5,774 개의 질의응답쌍으로 구성되어 있다.
- 공식 사이트
    - V1: https://korquad.github.io/category/1.0_KOR.html
    - V2: https://korquad.github.io/
- HuggingFace Dataset
    - https://huggingface.co/datasets/KorQuAD/squad_kor_v1


In [ ]:
from datasets import Dataset, load_dataset

ds = load_dataset("KorQuAD/squad_kor_v1", split="validation")

In [ ]:
refined_dict = {}

for data in ds:
  question = data['question']
  answer = data["answers"]["text"][0]
  refined_dict[question] = answer

len(refined_dict)

In [ ]:
count = 100
train_prompt_list = []
for idx, (question, answer) in enumerate(refined_dict.items()):
    if idx >= count:
        break

    prompt = f"<s>### Instruction: {question} ### Response: {answer}</s>"
    train_prompt_list.append({"text": prompt})

In [ ]:
train_dataset = Dataset.from_list(train_prompt_list)
train_dataset[-1]

In [ ]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments
)
from trl import SFTTrainer      
from peft import LoraConfig     

import os

In [ ]:
model_name = "TinyPixel/Llama-2-7B-bf16-sharded"

In [ ]:
load_in_4bit = True
bnb_4bit_compute_dtype = torch.float32
bnb_4bit_quant_type = "nf4"
bnb_4bit_use_double_quant = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=load_in_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=bnb_4bit_compute_dtype,
    bnb_4bit_use_double_quant=bnb_4bit_use_double_quant,
)

In [ ]:
# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1  # 병렬 처리를 단일 처리에 맞게 설정

In [ ]:
# tokenizer loading
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token  # padding 토큰과 eos 토큰을 같게 처리
tokenizer.padding_side = "right" # padding을 오른쪽(뒤에) 붙이도록 설정

In [ ]:
output_dir = "./results"
num_train_epochs = 40

bf16 = True

per_device_train_batch_size = 16
per_device_eval_batch_size = 16

learning_rate = 2e-4
weight_decay = 0.001

optim = "paged_adamw_32bit"

lr_scheduler_type = "constant"
warmup_ratio = 0.03

save_steps = 25
logging_steps = 25

In [ ]:
# Set training parameters
hub_model_name = '<계정>/llama-2-ko-7b-korquad'
api_key = "<API Key>"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",

    push_to_hub=True,          
    hub_model_id=hub_model_name,  
    hub_token=api_key,            
)

In [ ]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

# LoRA 설정
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

In [ ]:
max_seq_length = 256 

# Trainer 생성
trainer = SFTTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,  
    args=training_arguments,      
    peft_config=peft_config,      
)

In [ ]:
# 학습
from warnings import filterwarnings
filterwarnings(action='ignore')
trainer.train()

In [ ]:
import huggingface_hub

huggingface_hub.login(api_key)
tokenizer.push_to_hub(hub_model_name)

# 추론

##  Merge
- base model과 Lora 모델을 합쳐서 사용한다.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import PeftModel
import torch


model_name = "TinyPixel/Llama-2-7B-bf16-sharded"
hub_model_name = '<계정>/llama-2-ko-7b-korquad'

In [ ]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map="auto",
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

In [ ]:
load_model = PeftModel.from_pretrained(base_model, hub_model_name)
load_model = load_model.merge_and_unload() 

In [ ]:
def predict(query:str, pipeline)->str:
    prompt = f"<s>###Instruction: {query} ###Response: "
    result = pipeline(prompt)
    return result

In [ ]:
lora_pipeline = pipeline("text-generation", model=load_model, device_map="auto", tokenizer=tokenizer, max_length=256, truncation=True)


query = "임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?"
query = "헤이그가 군에서 퇴역한 년도는 몇년도입니까?"
predict(query, lora_pipeline)
